In [ ]:
reload_lamb()

In [ ]:
type_s = types.BasicType("s")
lang.get_system().add_basic_type(type_s)
intensional_types = meta.get_type_system()
intensional_types

# Intensional scope
### Notebook author: Kyle Rawlins / Apr 2018

This notebook goes through the basics of three approaches to scope in an intensional context. It is not intended as an introduction to the linguistic issues, but rather as a technical supplement to such an overview (which can be partially found in the von Fintel and Heim lecture notes). That is, this notebook presupposes familiarity with notions like "de re", "the third reading", etc.

The three approaches are:
* Intensionalization triggered by selection for an intensional type, binding all world variables in the scope of the selecting item (von Fintel and Heim lecture notes).
* World variables in the syntax (Percus 2000).
* Split intensionality (Keshet 2011).

I will focus on two main examples:

    (1) Joanna believes my brother is Canadian. (After one of Percus's examples)
    (2) Alfonso wants to buy an inexpensive coat. (After Keshet's version of Fodor's famous third reading example.)
    


# Scope with indices, intensionalization, and QR

We begin with the von Fintel & Heim first treatment of intensionalization. In this approach, selection of an intensional type triggers abstraction over the world of evaluation for the complement. I have implemented this here as a type shift feeding FA, rather than a separate composition operation.

In [ ]:
# set up evaluation world as a parameter to the interpretation function
int_system = lang.td_system.copy()
eval_w = lang.te("w_s")
int_system.assign_controller = lang.AssignmentController(specials=[eval_w])
lang.set_system(int_system)

# code to abstract over the evaluation world.
def intension_fun(f, assignment=None):
    new_a = lang.Assignment(assignment)
    new_a.update({"w": lang.te("w_s")})
    result = meta.LFun("w_s", f.content.under_assignment(new_a))
    return lang.UnaryComposite(f, result, source="INT(%s)" % (f.name))

int_system.add_rule(lang.UnaryCompositionOp("INT", intension_fun, typeshift=True))
int_system.typeshift = True
int_system

For the sake of being explicit about scope in these derivations, I will implement `my` using an existential quantifier.

In [ ]:
%%lamb
||brother|| = L x_e : L y_e : Brother(x,y,w)
||canadian|| = L x_e : Canadian(x,w)
||joanna|| = Joanna_e
||my|| = L f_<e,<e,t>> : L g_<e,t> : Exists x_e : f(Speaker_e)(x) & g(x)
dox = Dox_<(e,s),{s}>
||believe|| = L p_<s,t> : L x_e : Forall w2_s : w2 << dox(x, w_s) >> p(w2)

First, the basic unembedded sentence:

In [ ]:
(my * brother) * canadian

Surface scope, everything in situ:

In [ ]:
(joanna * (believe * ((my * brother) * canadian))) #.tree()

In [ ]:
trace = lang.Trace.index_factory(typ=types.type_e)
from lamb.lang import Binder

Wide scope (de re), DP has raised via QR (assuming this is possible across a finite clause boundary):

In [ ]:
((my * brother) *
 (Binder(11) *
  (joanna * (believe *
             (trace(11) * canadian))))) #.tree()

With just these tools, there is no other way to get another reading.

# World variables

This is the barest sketch of the world variable proposal in Percus 2000, *Constraints on some other variables in syntax*, Natural Language Semantics 8.

In [ ]:
w = lang.IndexedPronoun.index_factory("w", typ=type_s)

In [ ]:
# back to the basic composition system, just need to generalize PA
world_binding_system = lang.td_system.copy()

from lamb.types import TypeMismatch
from lamb.lang import BinaryComposite

# Predicate abstraction for world variables. Based on lang.sbc_pa
def world_pa(binder, content, assignment=None):
    if (binder.content is not None) or not binder.name.strip().isnumeric():
        raise TypeMismatch(binder, content, "Predicate Abstraction (type s)")
    index = int(binder.name.strip())
    vname = "var%i" % index
    bound_var = meta.term(vname, type_s)
    f = meta.LFun(type_s, content.content.calculate_partiality({bound_var}), vname)
    return BinaryComposite(binder, content, f)

world_binding_system.add_rule(lang.BinaryCompositionOp("WPA", world_pa, allow_none=True))
lang.set_system(world_binding_system)
world_binding_system

In [ ]:
%%lamb
||brother|| = L w_s : L x_e : L y_e : Brother(x,y,w)
||canadian|| = L w_s : L x_e : Canadian(x,w)
||joanna|| = Joanna_e
||my|| = L f_<e,<e,t>> : L g_<e,t> : Exists x_e : f(Speaker_e)(x) & g(x)
dox = Dox_<(e,s),{s}>
||believe|| = L w_s : L p_<s,t> : L x_e : Forall w2_s : w2 << dox(x, w_s) >> p(w2)

Basic unembedded sentence. As long as all world variables must be bound, and binders are only inserted up to interpretability to achieve a type $\langle s,t \rangle$ at the root, this is the only possible binding configuration for this sentence.

In [ ]:
Binder(1) * ((my * (brother * w(1))) * (canadian * w(1)))

Pure de dicto sentence:

In [ ]:
r = Binder(5) * (joanna *
                 ((believe * w(5)) *
                  (Binder(1) * ((my * (brother * w(1))) * (canadian * w(1))))))
r

In [ ]:
r.tree()

In principle there are several other binding configurations that are possible even without QR. Let's also assume a constraint against vacuous binding, with the effect that `Binder(1)` being inserted (required for semantic composition) forces at least one bound variable downstairs.

In [ ]:
r = Binder(5) * (joanna * ((believe * w(5)) *
                           (Binder(1) *
                            ((my * (brother * w(5))) * (canadian * w(1))))))
r

In [ ]:
r = Binder(5) * (joanna * ((believe * w(5)) * (Binder(1) * ((my * (brother * w(1))) * (canadian * w(5))))))
r

The second of these binding configurations would be banned by Percus's *Generalization X*. A slightly reformulated version of this condition: the world variable that a VP predicate selects most be coindexed with the nearest c-commanding binder.

With QR we would get another possibility. This binding configuration is really the only possibility for this scope configuration.

In [ ]:
r = Binder(5) * ((my * (brother * w(5))) *
                 (Binder(3) * (joanna * ((believe * w(5)) *
                                         (Binder(1) *
                                          (trace(3) * (canadian * w(1))))))))
r

# Split intensionality

This approach illustrates the core of the analysis in Keshet 2011, *Split intensionality: a new scope theory of de re and de dicto*, Linguistics and Philosophy 33. I will set things up slightly differently, implementing his intensionalization operation as a unary composition operation, rather than an operator. (This is mainly because it is a bit more straightforward in the lambda notebook, rather than for any deep reasons.)

In [ ]:
# set up evaluation world as a parameter to the interpretation function
int_system = lang.td_system.copy()
eval_w = lang.te("w_s")
int_system.assign_controller = lang.AssignmentController(specials=[eval_w])
lang.set_system(int_system)

# same intensionalization code as in the first version
def intension_fun(f, assignment=None):
    new_a = lang.Assignment(assignment)
    new_a.update({"w": lang.te("w_s")})
    result = meta.LFun("w_s", f.content.under_assignment(new_a))
    return lang.UnaryComposite(f, result, source="INT(%s)" % (f.name))

# add INT as a unary operation, *not* a type shift.
int_system.add_rule(lang.UnaryCompositionOp("INT", intension_fun))
int_system

There are two key ideas: (i) instead of generating propositions via a world binder inserted up to interpretability, we generate propositions with a shifting intensionalization operator (I will call this operator `INT`, Keshet notates it as ${}^{\wedge}$ after Montague). (ii) this intensionalization operator has syntactic presence, and QR can target a position above the operator or below it. The position above it generates transparent non-specific readings, but doesn't involve movement across a clause boundary.

In [ ]:
%%lamb
||brother|| = L x_e : L y_e : Brother(x,y,w)
||canadian|| = L x_e : Canadian(x,w)
||joanna|| = Joanna_e
||my|| = L f_<e,<e,t>> : Iota x_e : f(Speaker_e)(x) # note change from above -- discussed below
dox = Dox_<(e,s),{s}>
||believe|| = L p_<s,t> : L x_e : Forall w2_s : w2 << dox(x, w_s) >> p(w2)

The basic case is much the same as on the starting approach. To trigger unary composition, compose with `None`:

In [ ]:
((my * brother) * canadian) * None

The pure de dicto reading is not especially new or exciting; where before there was a type-shift, now we have an intensionalization operator:

In [ ]:
r = (joanna * (believe *
               (((my * brother) * canadian) * None)))
r

In [ ]:
r.tree()

What is new is the intermediate QR derivation:

In [ ]:
r = (joanna * (believe *
               ((my * brother) * (Binder(3) *
                                  ((trace(3) * canadian) * None)))))
r

And finally, the pure de re reading. There is no way to generate the reading that Percus needed to ban with generalization X, because intensional scope is still parasitic on DP scope.

In [ ]:
((my * brother) * (Binder(3) *
                   ((trace(3) * canadian) * None)))

One complication is that a regular generalized quantifier type won't work with regular FA on this approach, if it scopes above `INT`. You can see this by examining the type generated for a binder scoping over `INT`:

In [ ]:
Binder(3) * ((trace(3) * canadian) * None)

That is, an (extensional) generalized quantifier DP will be type $\langle \langle e,t \rangle , t \rangle$, but we really need something of type $\langle \langle e,\langle s,t\rangle \rangle , \langle s,t\rangle \rangle$. Keshet proposes a solution to this using a different composition operation for generalized quantifiers due to Daniel Buring; this operation also allows composition in object position without QR. This operation is rather complex and general, but for dealing with `INT` we really only need one special case, which I implement here as a binary composition via the following combinator. 'AS' stands for Argument Saturation.

In [ ]:
as_combinator = %te L quant_<<e,t>,t> : L f_<e,<X,t>> : L y_X : quant(L x_e : f(x)(y))
as_combinator

In [ ]:
#int_system.add_typeshift(as_combinator, "AS")
#int_system.typeshift = True
int_system.add_binary_rule(as_combinator, "AS")
int_system

In [ ]:
%%lamb
||my|| = L f_<e,<e,t>> : L g_<e,t> : Exists x_e : f(Speaker_e)(x) & g(x)

In [ ]:
r = (joanna * (believe *
               ((my * brother) * (Binder(3) *
                                  ((trace(3) * canadian) * None)))))
r

In [ ]:
r.tree()

In any case where QR above the intensional lexical item is possible, Keshet's system generates Fodor's three readings: a pure de dicto reading (non-specific/opaque), a pure de re reading (specific/transparent), and the third reading (non-specific/transparent). Here are the readings in an example more tailored to have them:

In [ ]:
%%lamb
||buy|| = L x_e : L y_e : Buy(y,x,w)
||coat|| = L x_e : Coat(x,w)
||inexpensive|| = L x_e : Inexpensive(x,w)
||want|| = L p_<s,t> : L x_e  : Forall w2_s : w2 << Des_<(e,s),{s}>(x,w) >> p(w2)
||alfonso|| = Alfonso_e
||an|| = L f_<e,t> : L g_<e,t> : Exists x_e : f(x) & g(x)

In [ ]:
PRO = lang.IndexedPronoun.index_factory("PRO")

Pure de dicto (non-specific / opaque), quantifier scopes under `INT`:

In [ ]:
r = alfonso * (Binder(10) *
               (trace(10) * (want *
                             (((an * (inexpensive * coat)) * (Binder(3) *
                                                              (PRO(10) * (buy * trace(3)))))
                              * None))))
r

Mixed (non-specific / transparent), quantifier scopes over `INT` but within clause:

In [ ]:
r = alfonso * (Binder(10) *
               (trace(10) * (want *
                             ((an * (inexpensive * coat)) * (Binder(3) *
                                                             ((PRO(10) * (buy * trace(3)))
                                                              * None))))))
r

Pure de re (specific/transparent), quantifier QRs all the way out across the non-finite clause boundary:

In [ ]:
r = ((an * (inexpensive * coat)) * (Binder(3) *
                                    (alfonso * (Binder(10) * (trace(10) *
                                                              (want *
                                                               ((PRO(10) * (buy * trace(3)))
                                                                * None)))))))
r

# Fourth readings

None of the systems above generate the so-called fourth reading: specific/opaque. This is because with QR as the scope mechanism, the QR target position provides a lower bound on the world binding, and this kind of reading would require a world binding that is local, but a quantifier position that is not. There has been some debate about whether the fourth reading exists, so perhaps this is a virtue: see Keshet 2011 footnotes 4,15 for an argument along these lines. However, Szabo in a 2010 Amsterdam Colloquium paper (*Specific, yet opaque*) provides fairly convincing cases of the fourth reading.

If you do want to derive this reading, there are various options.
* Szabo: add a new QR operation that moves only the determiner.
* Do partial reconstruction of the NP from a pure de re reading into a position where the world variable for the NP gets locally bound. (For Percus, under the world binder, for Keshet or a type-shifting approach, under the `INT` operator.)

Both of these are a bit convoluted. Here's a sketch of what Szabo's new QR operation would do. This operation moves a quantifier with a silent `thing` restrictor, and leaves a trace that gets interpreted via a variant (of sorts) of Fox's Trace Conversion rule. In Szabo's system, there is a specialized composition rule for this, but for the sake of simplicity, I just hard-code this in a specialized pre-indexed trace entry `rt3`.

Unfortunately some of the simplification involved in this use of Iota is not currently implemented in the lambda notebook.

In [ ]:
%%lamb
||thing|| = L x_e : True
||rt3|| = L f_<e,t> : Iota x_e: f(x) & x==var3

First, it may be helpful to see a derivation without an intensional lexical item where this rule has applied. In the result, the formula will be false for any potential witnesses for the existential that don't verify the properties under the $\iota$ operator, leading to equivalence with the standard truth-conditions. (This is not something that the lambda notebook knows how to simplify for you.)

In [ ]:
((an * thing) * (Binder(3) * (alfonso * (buy * (rt3 * (inexpensive * coat))))))

Here is a fourth reading derivation, using Keshet's intensionalization operator from above.

In [ ]:
((an * thing) * (Binder(3) * (alfonso * (Binder(10) *
                                         (trace(10) * (want * ((PRO(10) *
                                                               (buy * (rt3 * (inexpensive * coat))))
                                                               * None)))))))

Basically, the $\iota$ condition on $x$s requires that any witness for the outer quantifier satisfy the properties inside the $\iota$ condition, and so the formula is only true if there are values of $x$ that do satisfy these conditions. (Is it false or undefined if not? As I understand it, the intent is that it would be false. The Russellian model-theoretic interpretation for inverted $\iota$ in, e.g., Gamut vol 1 sec 5.2 would derive the result needed.)

In conclusion: I hope that if fourth readings are real, there is another way to derive them.